# MSCF v2.2.0: Reproducing Key Results

This notebook reproduces the numerical results, figures, and tables from:

> Roland, I. (2026). *Matter-Space Coupling Framework: Deriving General Relativity as the IR Limit of Causal Substrate Dynamics* (MSCF v2.2.0).

All computations are self-contained. Heavy results (transfer function, phase correlations) are loaded from precomputed data in `results/`. Lightweight results (echo delays, ghost window, greybody factors) are computed inline.

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.integrate import solve_ivp

REPO_ROOT = Path('.').resolve().parent
RESULTS = REPO_ROOT / 'results'
FIGURES = REPO_ROOT / 'paper' / 'figures'

plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'figure.dpi': 150,
    'lines.linewidth': 1.5,
    'mathtext.fontset': 'cm',
})

---
## 1. Echo Delay Table (Table II, Section XI)

The echo delay is determined by the Kerr tortoise coordinate distance between the light ring and the MSCF barrier at $r_b = M$:

$$\Delta t_{\text{echo}}(M, \chi) = \frac{2}{c} |r_*(r_{\text{lr}}(\chi)) - r_*(M)|$$

This is a deterministic function of remnant mass and spin with no free parameters.

In [ ]:
def light_ring_radius(chi):
    """Co-rotating equatorial light ring radius in units of M."""
    chi = np.clip(chi, -1 + 1e-15, 1 - 1e-15)
    return 2 * (1 + np.cos(2/3 * np.arccos(-abs(chi))))


def kerr_rstar(r, chi):
    """Kerr tortoise coordinate r*(r) in units of M."""
    chi = np.clip(chi, -1 + 1e-15, 1 - 1e-15)
    s = np.sqrt(1 - chi**2)
    rp = 1 + s  # outer horizon
    rm = 1 - s  # inner horizon
    d = rp - rm
    if abs(d) < 1e-14:  # Schwarzschild limit
        return r + 2 * np.log(abs(r/2 - 1))
    Ap = 2 * rp / d
    Am = 2 * rm / d
    return r + Ap * np.log(abs((r - rp) / 2)) - Am * np.log(abs((r - rm) / 2))


def echo_delay_over_M(chi):
    """Echo delay dt/M = 2|r*(r_lr) - r*(r_b)| with r_b = M."""
    r_lr = light_ring_radius(chi)
    return 2 * abs(kerr_rstar(r_lr, chi) - kerr_rstar(1.0, chi))


# Table II values from the paper
chi_table = [0.0, 0.5, 0.9, 0.99, 0.999]

print('Table II: Predicted Echo Delays (MSCF v2.2.0, Section XI)')
print(f'{"chi":>6}  {"r_lr/M":>8}  {"dt/M_time":>10}')
print('-' * 30)
for chi in chi_table:
    r_lr = light_ring_radius(chi)
    dt = echo_delay_over_M(chi)
    print(f'{chi:6.3f}  {r_lr:8.3f}  {dt:10.3f}')

In [ ]:
# Plot echo delay vs spin
chi_fine = np.linspace(0, 0.999, 200)
dt_fine = [echo_delay_over_M(c) for c in chi_fine]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(chi_fine, dt_fine, 'C0-', lw=2)
ax.set_xlabel(r'Spin $\chi$')
ax.set_ylabel(r'$\Delta t_{\rm echo}\, /\, M_{\rm time}$')
ax.set_title('Echo delay vs spin (MSCF v2.2.0)')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 2. Transfer Function and Bogoliubov Phase (Figure 1, Section IX)

The coupled 2x2 mode evolution through the bounce produces a transfer function $T^2(k)$ and a Bogoliubov phase $\arg(\alpha_k)$. The bounce produces ~330 degrees of phase variation at $k > 10^{-2}$ (Planck units), but CMB modes at $k < 10^{-4}$ lie in the asymptotic plateau where both amplitude and phase are constant to < 0.2%.

Data loaded from precomputed results of `coupled_mode_evolution.py` and `phase_correlation_test.py`.

In [ ]:
# Load precomputed data
phase_dat = np.loadtxt(RESULTS / 'cmb_comparison' / 'phase_function.dat')
k_phase = phase_dat[:, 0]
arg_alpha = phase_dat[:, 1]

with open(RESULTS / 'cmb_comparison' / 'phase_results.json') as f:
    phase_json = json.load(f)

with open(RESULTS / 'transfer_functions' / 'full_coupled_results.json') as f:
    coupled_json = json.load(f)

k_coupled = np.array([m['k'] for m in coupled_json['coupled']])
T2_coupled = np.array([m['T2'] for m in coupled_json['coupled']])
k_uncoupled = np.array([m['k'] for m in coupled_json['uncoupled']])
T2_uncoupled = np.array([m['T2'] for m in coupled_json['uncoupled']])

print(f'Loaded {len(k_phase)} phase points, {len(k_coupled)} coupled T^2 points')
print(f'Low-k T^2 coupled/uncoupled ratio: {T2_coupled[0]/T2_uncoupled[0]:.3f} (32% suppression)')

In [ ]:
# Figure 1: Transfer function and Bogoliubov phase (4-panel summary)
unc_modes = phase_json.get('mode_results_uncoupled', [])
k_unc = np.array([m['k'] for m in unc_modes])
arg_alpha_unc = np.array([m['arg_alpha'] for m in unc_modes])
mp_data = phase_json['multipole_phases']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# (a) Phase across full k range
ax = axes[0, 0]
ax.plot(k_phase, arg_alpha, 'C0-', lw=2, label='Coupled')
ax.plot(k_unc, arg_alpha_unc, 'C1--', lw=1.5, label='Uncoupled')
ax.axvspan(1e-8, 1e-4, alpha=0.07, color='steelblue')
ax.axvspan(0.01, 10, alpha=0.07, color='firebrick')
ax.set_xscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$\arg(\alpha_k)$ [rad]')
ax.set_title(r'(a) Bogoliubov phase $\arg(\alpha_k)$')
ax.legend(loc='lower left', fontsize=8)
ax.set_xlim(1e-8, 20)
ax.set_ylim(-3.5, 2.8)
ax.grid(True, alpha=0.2)
ax.text(3e-7, 2.3, 'CMB scales', color='steelblue', fontsize=9, ha='center', fontweight='bold')
ax.text(0.3, 2.3, r'$\Xi$-well region', color='firebrick', fontsize=9, ha='center', fontweight='bold')
ax.annotate(r'$\Delta\phi \approx 330°$', xy=(0.1, -2.8), fontsize=9, color='firebrick', ha='center',
            bbox=dict(boxstyle='round,pad=0.2', fc='white', ec='firebrick', alpha=0.7))

# (b) CMB zoom
ax = axes[0, 1]
cmb_mask = k_phase <= 1e-4
ax.plot(k_phase[cmb_mask], arg_alpha[cmb_mask], 'C0-', lw=2.5, label='Coupled')
unc_cmb = k_unc <= 1e-4
ax.plot(k_unc[unc_cmb], arg_alpha_unc[unc_cmb], 'C1--', lw=1.8, label='Uncoupled')
ax.set_xscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$\arg(\alpha_k)$ [rad]')
ax.set_title(r'(b) CMB range: $\Delta\phi = 2.9\times10^{-3}$ rad')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.2)

# (c) Multipole phases
ax = axes[1, 0]
for kappa_str, label, color, ms in [
        ('0.000316', r'$\kappa = 3.16\times10^{-4}$', 'C0', 'o'),
        ('0.000562', r'$\kappa = 5.62\times10^{-4}$', 'C1', 's'),
        ('0.001', r'$\kappa = 1.00\times10^{-3}$', 'C2', '^')]:
    mp = mp_data[kappa_str]
    ells = sorted([int(e) for e in mp.keys()])
    phi_vals = np.array([mp[str(e)]['phi_arg_alpha'] for e in ells])
    phi_rel = phi_vals - phi_vals[0]
    ax.plot(ells, np.degrees(phi_rel) * 3600, f'{ms}-', markersize=3, label=label, color=color, lw=1.5)
ax.set_xlabel(r'Multipole $\ell$')
ax.set_ylabel(r'$\phi_\ell - \phi_{\ell=2}$ [arcsec]')
ax.set_title('(c) Multipole phase variation')
ax.legend(fontsize=8)
ax.set_xlim(1, 31)
ax.grid(True, alpha=0.2)

# (d) Coupling ratio
ax = axes[1, 1]
n = min(len(k_coupled), len(k_uncoupled))
ratio = T2_coupled[:n] / T2_uncoupled[:n]
ax.plot(k_coupled[:n], ratio, 'C3o-', ms=5, lw=2)
ax.axhline(1.0, color='gray', ls='--', lw=0.8)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$T^2_{\\rm coupled} / T^2_{\\rm uncoupled}$')
ax.set_title('(d) Landau-Zener coupling ratio')
ax.set_ylim(1e-3, 2)
ax.grid(True, alpha=0.2)
ax.annotate(f'Low-$k$ plateau: {ratio[0]:.3f}\n(32% suppression)',
            xy=(k_coupled[2], ratio[2]), xytext=(0.03, 0.12),
            arrowprops=dict(arrowstyle='->', color='C3', lw=1.2),
            fontsize=8, color='C3',
            bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='C3', alpha=0.8))

fig.suptitle('Figure 1: Transfer function and Bogoliubov phase (MSCF v2.2.0)', fontsize=13, y=1.01)
fig.tight_layout()
plt.show()

---
## 3. Two-Surface Interior Structure (Figure 2, Section X)

The MSCF interior metric function is derived from the constitutive response $F(x) = 1 - x$, yielding the resistance factor $\Omega(x_g) = 2 - x_g$ and the metric:

$$f(r) = (1 - x_g)(2 - x_g), \qquad x_g = r_s / r$$

This has two zeros:
- The **horizon** at $r = r_s$ ($x_g = 1$): classically transparent ($C^1$ continuous with Schwarzschild)
- The **inversion barrier** at $r = r_s/2$ ($x_g = 2$): total reflectivity

The metric is $C^1$ at the horizon: $F'(x_g = 1) = -1$, matching the Schwarzschild derivative with no kink.

In [ ]:
from scipy.integrate import quad

def cavity_integrand(r_over_rs):
    """Proper distance integrand dr / sqrt(|F(r)|), with r in units of r_s."""
    x_g = 1.0 / r_over_rs  # x_g = r_s / r
    F_abs = abs((1 - x_g) * (2 - x_g))
    return 1 / np.sqrt(F_abs) if F_abs > 1e-14 else 0.0

# Cavity between horizon (r = r_s, x_g = 1) and barrier (r = r_s/2, x_g = 2)
delta_ell_rs, _ = quad(cavity_integrand, 0.501, 0.999)
print(f'Cavity proper length: {delta_ell_rs:.4f} r_s')

# Plot metric function
u = np.linspace(0.32, 3.5, 2000)
x_g = 1.0 / u  # x_g = r_s / r, so r/r_s = u = 1/x_g
F_mscf = (1 - x_g) * (2 - x_g)
f_gr = 1 - x_g  # Schwarzschild

# Verify C1 continuity: F'(x_g) = -3 + 2*x_g, so F'(1) = -1 = f_gr'(1)
dF_at_horizon = -3 + 2 * 1.0
df_gr_at_horizon = -1.0
print(f'F\'(x_g=1) = {dF_at_horizon:.1f}, f_GR\'(x_g=1) = {df_gr_at_horizon:.1f}  (C1 match: {dF_at_horizon == df_gr_at_horizon})')

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(u, F_mscf, 'k-', lw=2.5, label=r'MSCF: $F = (1-x_g)(2-x_g)$')
ax.plot(u, f_gr, color='#888', ls='--', lw=1.8, alpha=0.75, label=r'GR: $f = 1 - x_g$')
ax.axhline(0, color='black', lw=0.6)

# Mark zeros
ax.plot(1.0, 0, 'ko', ms=9, markeredgecolor='white', markeredgewidth=1.5)
ax.plot(0.5, 0, 'ks', ms=9, markeredgecolor='white', markeredgewidth=1.5)

# Mark minimum
x_g_min = 1.5  # d/dx_g[(1-x)(2-x)] = -3+2x = 0 => x_g = 3/2
r_min = 1.0 / x_g_min
F_min = (1 - x_g_min) * (2 - x_g_min)
ax.plot(r_min, F_min, 'kv', ms=7, zorder=10)
ax.annotate(f'$F_{{\\min}} = -1/4$', xy=(r_min, F_min), xytext=(0.9, -0.35),
            arrowprops=dict(arrowstyle='->', lw=1.0), fontsize=9, ha='center')

ax.annotate('Horizon\n$r = r_s$ ($x_g = 1$)\nclassically transparent ($C^1$)',
            xy=(1.0, 0), xytext=(1.6, -0.18),
            arrowprops=dict(arrowstyle='->', lw=1.2), fontsize=9, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='#888', alpha=0.95))
ax.annotate('Inversion barrier\n$r = r_s/2$ ($x_g = 2$)',
            xy=(0.5, 0), xytext=(0.75, 0.32),
            arrowprops=dict(arrowstyle='->', lw=1.2), fontsize=9, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='#888', alpha=0.95))

ax.set_xlabel(r'$r / r_s$')
ax.set_ylabel('Metric function')
ax.set_title(r'Figure 2: MSCF metric $F(r) = (1-x_g)(2-x_g)$ vs Schwarzschild (v2.2.0)')
ax.set_xlim(0.32, 3.5)
ax.set_ylim(-0.45, 0.65)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.12)
plt.tight_layout()
plt.show()

---
## 4. Echo Amplitude Trains (Figure 3, Section XI)

**v2.2.0 update:** The echo amplitude hierarchy is now derived from greybody factors with no free parameters.

The angular momentum barrier is *nearly transparent* at the QNM frequency ($M\omega \approx 0.52$):
$|R_b|^2 = 0.008$, $|T_b|^2 = 0.992$. This gives $A_1/A_2 \approx 11.5$.

Three models are compared:
1. **Generic ECO** (single reflective surface): monotonic geometric decay
2. **Classical MSCF** (primary prediction): strong first echo, steep drop — derived from $A_n(\omega) = |T_b| \cdot |R_b|^{n-1}$
3. **Quantum MSCF** (secondary possibility): amplitude inversion $A_2 > A_1$ if horizon acquires reflectivity $R_1 > 0$ from quantum corrections

In [ ]:
N_echoes = 8
n = np.arange(1, N_echoes + 1)

# 1. Generic single-surface ECO: monotonic geometric decay
R_generic = 0.65
A_generic = R_generic ** (n - 1)

# 2. Classical MSCF: derived greybody factors at QNM
Tb = 0.996   # barrier transmissivity at QNM (from Regge-Wheeler solver)
Rb = 0.087   # barrier reflectivity at QNM
A_classical = Tb * Rb ** (n - 1)

# 3. Quantum MSCF: horizon acquires reflectivity R_1 > 0
R1_quantum = 0.15
T1_quantum = 1.0 - R1_quantum
A_quantum = np.zeros(N_echoes)
A_quantum[0] = R1_quantum
A_quantum[1] = T1_quantum**2
for i in range(2, N_echoes):
    A_quantum[i] = A_quantum[1] * R1_quantum ** (i - 1)

# Normalize for comparison
A_generic_norm = A_generic / A_generic[0]
A_classical_norm = A_classical / A_classical[0]
A_quantum_norm = A_quantum / A_quantum.max()

print(f'Classical MSCF: A1/A2 = {A_classical[0]/A_classical[1]:.1f}')
print(f'  |R_b|^2 = {Rb**2:.3f}, |T_b|^2 = {Tb**2:.3f}')

fig, ax = plt.subplots(figsize=(8, 5.5))

C_GEN = '#888888'
C_CLASS = '#2266AA'
C_QUANT = '#D94A4A'

ax.plot(n, A_generic_norm, 'o--', color=C_GEN, ms=10, lw=1.8,
        markeredgecolor='white', markeredgewidth=1.2,
        label='Generic ECO (single surface)', zorder=5)

ax.plot(n, A_classical_norm, 's-', color=C_CLASS, ms=11, lw=2.5,
        markeredgecolor='white', markeredgewidth=1.2,
        label=r'Classical MSCF ($|R_b|^2 = 0.008$)', zorder=7)

ax.plot(n, A_quantum_norm, 'D--', color=C_QUANT, ms=9, lw=1.8,
        markeredgecolor='white', markeredgewidth=1.2, alpha=0.75,
        label=r'Quantum MSCF ($R_1 > 0$, if present)', zorder=6)

ax.annotate(r'$A_1 / A_2 \approx 11$',
            xy=(2, A_classical_norm[1]),
            xytext=(3.2, 0.38),
            arrowprops=dict(arrowstyle='->', color=C_CLASS, lw=1.5,
                            connectionstyle='arc3,rad=0.2'),
            fontsize=11, ha='center', va='center', color=C_CLASS,
            fontweight='bold', zorder=20,
            bbox=dict(boxstyle='round,pad=0.3', fc='#E8F0FF', ec=C_CLASS,
                      alpha=0.95, lw=1.2))

ax.annotate(r'$A_2 > A_1$',
            xy=(1.5, (A_quantum_norm[0] + A_quantum_norm[1]) / 2),
            xytext=(1.1, 0.55),
            arrowprops=dict(arrowstyle='->', color=C_QUANT, lw=1.2,
                            connectionstyle='arc3,rad=0.3'),
            fontsize=10, ha='center', va='center', color=C_QUANT,
            fontweight='bold', alpha=0.75, zorder=20,
            bbox=dict(boxstyle='round,pad=0.25', fc='#FFF0F0', ec=C_QUANT,
                      alpha=0.7, lw=1.0))

ax.set_xlabel('Echo number  ' + r'$n$', fontsize=14)
ax.set_ylabel('Relative amplitude  ' + r'$A_n / A_{\max}$', fontsize=14)
ax.set_xlim(0.4, N_echoes + 0.6)
ax.set_ylim(-0.05, 1.15)
ax.set_xticks(n)
ax.set_title('Figure 3: Echo amplitude trains (MSCF v2.2.0)')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.15)
plt.tight_layout()
plt.show()

---
## 5. Ghost Window Analysis (Section IX)

The ghost window is where the matter eigenvalue $G_{\rm matt} < 0$. It is benign: sub-Planckian in duration, invisible to CMB modes.

In [ ]:
with open(RESULTS / 'ghost_analysis' / 'ghost_window.json') as f:
    ghost = json.load(f)

print('Ghost Window Analysis (MSCF v2.2.0, Section IX)')
print(f'  Duration (cosmic time):   {ghost["Delta_t_ghost_planck"]:.4f} t_Pl')
print(f'  Duration (conformal):     {ghost["Delta_eta_ghost_planck"]:.4f} eta_Pl')
print(f'  Resonance threshold:      k > {ghost["k_threshold"]:.4f} (Planck units)')
print(f'  CMB modes:                k * Delta_eta ~ {ghost["k_CMB_ratio"]}')
print(f'  Verdict:                  {ghost["verdict"]}')

---
## 6. Planck Commander Likelihood (Section IX)

Comparison against the Planck 2018 Commander TT likelihood at $\ell = 2$-$29$. LCDM is preferred at all tested values of the scale parameter $\kappa$.

In [ ]:
with open(RESULTS / 'cmb_comparison' / 'commander_results.json') as f:
    commander = json.load(f)

print('Planck Commander Likelihood (MSCF v2.2.0, Section IX)')
print(f'  logL(LCDM) = {commander["results"][0]["logL_lcdm"]:.2f}')
print()
for entry in commander['results']:
    kappa = entry['kappa']
    dlogL = entry['delta_logL']
    print(f'  kappa = {kappa:.2e}: Delta logL = {dlogL:.2f} (LCDM preferred)')

---
## 7. Algebraic Identities (Sections VI and IX)

Two algebraic results from the paper verified numerically:

**Theorem 6.4 (Self-Consistency):** $\xi / \eta^2 = 1$ for any $\ell_* / \ell_P$, where $\eta = \ell_*^2 / \ell_P^2$ (entropy per link) and $\xi = \ell_*^4 / \ell_P^4$ (max links per 4-cell).

**Theorem 9.8 (Substrate-Matter Unity):** $F_s^{\rm total} / M^2 = \varepsilon_m - 1$ where $\varepsilon_m = 3(1+w)/2$. Stability requires $w > -1/3$.

In [ ]:
# Theorem 6.4: Self-Consistency Identity
G = 6.67430e-11
c_light = 2.99792458e8
hbar = 1.054571817e-34
l_P = np.sqrt(hbar * G / c_light**3)
rho_crit = c_light**7 / (hbar * G**2)

print('Theorem 6.4: xi / eta^2 = 1 for all l_*')
print(f'  {"l_*/l_P":>10}  {"eta":>14}  {"xi":>14}  {"xi/eta^2":>16}')
print('  ' + '-' * 60)
for r in [0.37, 0.5, 1.0, np.sqrt(2), 2.0]:
    l_star = r * l_P
    eta = c_light**3 * l_star**2 / (hbar * G)
    xi = rho_crit * l_star**4 / (hbar * c_light)
    print(f'  {r:10.4f}  {eta:14.6e}  {xi:14.6e}  {xi / eta**2:16.10f}')

print()

# Theorem 9.8: Substrate-Matter Unity
print('Theorem 9.8: F_s^total / M^2 = eps_m - 1 = 3(1+w)/2 - 1')
print(f'  {"Matter":>14}  {"w":>6}  {"eps_m":>8}  {"F_s/M^2":>10}  {"Stable":>8}')
print('  ' + '-' * 52)
for name, w in [('Dust', 0.0), ('Radiation', 1/3), ('Stiff matter', 1.0), ('Dark energy', -1.0)]:
    eps_m = 3 * (1 + w) / 2
    fs = eps_m - 1
    print(f'  {name:>14}  {w:>6.2f}  {eps_m:>8.3f}  {fs:>+10.3f}  {"yes" if fs > 0 else "no":>8}')

---
## 8. Remnant Properties (Section XII)

MSCF predicts a minimum black hole mass $M_{\rm min} = M_P / 2$ (Eq. 39) and a modified Hawking temperature $T_{\rm eff} = T_H [1 - \frac{1}{4}(M_P/M)^2]$ (Eq. 43) that vanishes at $M_{\rm min}$, yielding stable Planck-mass remnants.

The gravitational scattering cross-section of these remnants is $\sigma_n \sim 10^{-60}$ cm$^2$ per nucleon -- roughly 30 orders of magnitude below the best direct detection limits. See `mscf_remnant_detection/` for the full analysis.

In [ ]:
# Remnant derivation chain (Eqs. 38-43)
k_B = 1.380649e-23
M_P = np.sqrt(hbar * c_light / G)

# Eq. 38: kappa_max
kappa_max = c_light**2 / (2 * l_P)

# Eq. 39: M_min = M_P / 2
M_min = M_P / 2

# Eq. 43: T_eff = T_H [1 - (1/4)(M_P/M)^2]
def T_eff(M):
    T_H = hbar * c_light**3 / (8 * np.pi * G * M * k_B)
    return T_H * (1 - 0.25 * (M_P / M)**2)

# Verify T_eff(M_P/2) = 0
correction_at_Mmin = 1 - 0.25 * (M_P / M_min)**2

print('Remnant Properties (MSCF v2.2.0, Section XII)')
print(f'  M_P          = {M_P:.6e} kg')
print(f'  M_min = M_P/2 = {M_min:.6e} kg = {M_min / 1.78266192e-27:.3e} GeV')
print(f'  kappa_max    = {kappa_max:.6e} m/s^2')
print(f'  r_s(M_min)   = {2*G*M_min/c_light**2:.3e} m = {2*G*M_min/(c_light**2 * l_P):.6f} l_P')
print()
print('  T_eff at various masses:')
for factor in [100, 10, 2, 1, 0.5]:
    M = factor * M_P
    T = T_eff(M)
    print(f'    M = {factor:6.1f} M_P:  T_eff = {T:.3e} K')
print()
print(f'  Critical check: correction at M_P/2 = {correction_at_Mmin:.2e} (should be 0)')
print(f'  Gate 0 PASS: {abs(correction_at_Mmin) < 1e-15}')

---
## 9. Greybody-Derived Echo Amplitudes (New in v2.2.0, Section XI)

The angular momentum barrier reflectivity $R_b(\omega)$ is computed from the Regge-Wheeler scattering problem — a standard GR calculation with no free parameters. At the QNM frequency of a stellar-mass BBH ($M\omega \approx 0.52$), the barrier is *nearly transparent*.

Here we reproduce the greybody computation inline (self-contained, no external dependencies).

In [ ]:
def regge_wheeler_potential(r, l=2):
    """Regge-Wheeler potential V(r) in units where 2M = 1 (r_s = 1)."""
    return (1 - 1/r) * (l*(l+1)/r**2 - 3/(r**3))


def rstar_from_r(r):
    """Tortoise coordinate r* from r, in units where r_s = 1."""
    return r + np.log(np.abs(r - 1))


def greybody_factor(omega, l=2, rstar_min=-50, rstar_max=80):
    """Compute |R_b|^2 and |T_b|^2 from Regge-Wheeler scattering.
    
    Shoot from near-horizon with purely ingoing BC: psi = exp(-i omega r*).
    Extract A_in, A_out at large r* from the outgoing decomposition.
    """
    from scipy.interpolate import CubicSpline
    
    # Build potential on r* grid
    r_grid = np.linspace(1.001, 200, 10000)
    rs_grid = rstar_from_r(r_grid)
    V_grid = np.array([regge_wheeler_potential(r, l) for r in r_grid])
    V_interp = CubicSpline(rs_grid, V_grid, extrapolate=True)
    
    # Find adaptive start: where V/omega^2 < 1e-10
    rs_start = rstar_min
    for rs_test in np.linspace(rstar_min, -5, 200):
        if abs(V_interp(rs_test)) / max(omega**2, 1e-30) < 1e-10:
            rs_start = rs_test
            break
    
    # IC: purely ingoing at horizon: psi = exp(-i omega r*)
    y0 = [np.cos(-omega * rs_start),            # Re(psi)
          np.sin(-omega * rs_start),             # Im(psi)
          omega * np.sin(-omega * rs_start),     # Re(dpsi/dr*)
          -omega * np.cos(-omega * rs_start)]    # Im(dpsi/dr*)
    
    def rhs(rs, y):
        pr, pi, dpr, dpi = y
        V = float(V_interp(rs))
        return [dpr, dpi, (V - omega**2) * pr, (V - omega**2) * pi]
    
    wavelength = 2 * np.pi / max(omega, 0.01)
    max_step = min(wavelength / 10, (rstar_max - rs_start) / 500)
    
    sol = solve_ivp(rhs, [rs_start, rstar_max], y0,
                    method='DOP853', rtol=1e-10, atol=1e-12, max_step=max_step)
    
    # Extract at r*_max and decompose into A_in, A_out
    psi = complex(sol.y[0, -1], sol.y[1, -1])
    dpsi = complex(sol.y[2, -1], sol.y[3, -1])
    rs_end = rstar_max
    
    A_in = (omega * psi + 1j * dpsi) * np.exp(1j * omega * rs_end) / (2 * omega)
    A_out = (omega * psi - 1j * dpsi) * np.exp(-1j * omega * rs_end) / (2 * omega)
    
    Tb2 = 1.0 / abs(A_in)**2
    Rb2 = abs(A_out)**2 / abs(A_in)**2
    
    return {'Tb2': Tb2, 'Rb2': Rb2, 'flux_err': abs(Tb2 + Rb2 - 1.0),
            'Tb': np.sqrt(Tb2), 'Rb': np.sqrt(Rb2)}


# Compute greybody spectrum
Momega_arr = np.linspace(0.05, 2.0, 100)
Rb2_arr = np.zeros_like(Momega_arr)
Tb2_arr = np.zeros_like(Momega_arr)

print('Computing greybody factors (Regge-Wheeler, l=2)...')
for i, Mw in enumerate(Momega_arr):
    omega = Mw * 2  # convert Momega to omega (2M = 1 in our units)
    g = greybody_factor(omega, l=2)
    Rb2_arr[i] = g['Rb2']
    Tb2_arr[i] = g['Tb2']

# Values at GW150914 QNM: Momega = 0.523
Mw_qnm = 0.523
g_qnm = greybody_factor(Mw_qnm * 2, l=2)
print(f'\nAt GW150914 QNM (Momega = {Mw_qnm}):')
print(f'  |R_b|^2 = {g_qnm["Rb2"]:.4f}')
print(f'  |T_b|^2 = {g_qnm["Tb2"]:.4f}')
print(f'  Flux conservation error = {g_qnm["flux_err"]:.2e}')
print(f'  A_1 = {g_qnm["Tb"]:.4f}')
print(f'  A_2 = {g_qnm["Tb"] * g_qnm["Rb"]:.4f}')
print(f'  A_1/A_2 = {1/g_qnm["Rb"]:.1f}')

peak_idx = np.argmax(Rb2_arr)
print(f'\nBarrier peak at Momega = {Momega_arr[peak_idx]:.2f}: |R_b|^2 = {Rb2_arr[peak_idx]:.3f}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.plot(Momega_arr, Tb2_arr, 'C0-', lw=2, label=r'$|T_b|^2$')
ax1.plot(Momega_arr, Rb2_arr, 'C3-', lw=2, label=r'$|R_b|^2$')
ax1.axvline(Mw_qnm, color='gray', ls=':', lw=1, label='GW150914 QNM')
ax1.set_xlabel(r'$M\omega$')
ax1.set_ylabel('Transmission / Reflection')
ax1.set_title('Greybody factors (Regge-Wheeler, l=2)')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.2)
ax1.annotate(f'$|R_b|^2 = {g_qnm["Rb2"]:.3f}$\nat QNM',
             xy=(Mw_qnm, g_qnm['Rb2']), xytext=(0.8, 0.3),
             arrowprops=dict(arrowstyle='->', lw=1.2, color='C3'),
             fontsize=10, color='C3')

N_echo = 6
n_echo = np.arange(1, N_echo + 1)
A_n = g_qnm['Tb'] * g_qnm['Rb'] ** (n_echo - 1)

ax2.bar(n_echo, A_n, color='#2266AA', edgecolor='white', linewidth=1.5, width=0.6)
ax2.set_xlabel('Echo number $n$')
ax2.set_ylabel(r'$A_n = |T_b| \cdot |R_b|^{n-1}$')
ax2.set_title(f'Derived echo amplitudes (GW150914, $M\\omega = {Mw_qnm}$)')
ax2.set_xticks(n_echo)
ax2.grid(True, alpha=0.2, axis='y')
for i, a in enumerate(A_n):
    if a > 0.01:
        ax2.text(n_echo[i], a + 0.02, f'{a:.3f}', ha='center', fontsize=9, fontweight='bold')

fig.suptitle('Section X: Zero-parameter echo amplitudes from greybody factors (MSCF v2.2.0)', fontsize=12, y=1.02)
fig.tight_layout()
plt.show()

---

All results above are consistent with MSCF v2.2.0. The cosmological sector (Section IX) shows null departures from LCDM due to scale separation. The primary falsifiable prediction is the gravitational wave echo signature (Section XI), now with zero-free-parameter greybody-derived amplitudes. The barrier is nearly transparent at the QNM frequency ($|R_b|^2 \approx 0.008$), yielding a strong first echo with steep decay.

For the full derived-parameter echo search results (4 events, 9 detectors, matched filter + cepstrum, 5/5 null tests), see `mscf_derived_echo/results/RESULTS.md`.